In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import zipfile
import datetime
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re

In [2]:
df = pd.read_csv('cleaned_dataset.csv', encoding='iso-8859-1')
df.head()

,subject,from,date,to,label,thread,name,date_utc,time,hour,month,num_qm,num_em,hashtag
0,Complete your setup with the Slack desktop app,"""Slack"" <no-reply@email.slackhq.com>",2021-01-21 11:02:05-06:00,<tanishkumar@berkeley.edu>,"Inbox,Important,Category Promotions,Unread",1.689517e+18,Slack,2021-01-21 17:02:05+00:00,11:02:05,11,1,0,0,0
1,General Registration is now open for Winter/Sp...,Mission College <nadler@wvm.edu>,2020-12-07 13:45:42-05:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.685446e+18,Mission College,2020-12-07 18:45:42+00:00,13:45:42,13,12,0,1,0
2,A global bank that doubles as a cybersecurity ...,"""Tricia at WayUp"" <info@bb3.wayup.com>",2020-12-08 02:19:45+00:00,"""Tanish Kumar"" <tanishkumar@berkeley.edu>","Inbox,Opened,Category Promotions",1.685475e+18,Tricia at WayUp,2020-12-08 02:19:45+00:00,02:19:45,2,12,1,1,0
3,The next biggest thing,DailyDropout.fyi <dailydropout@substack.com>,2021-01-29 23:09:33+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.690265e+18,DailyDropout.fyi,2021-01-29 23:09:33+00:00,23:09:33,23,1,0,0,0
4,Read free for 30 days,Scribd <hello@hello.scribd.com>,2021-02-18 08:51:55+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.692022e+18,Scribd,2021-02-18 08:51:55+00:00,08:51:55,8,2,0,0,0


In [3]:
# The proportion of upper-case letters in the whole string
df['prop_upper'] = df['subject'].str.count(r'[A-Z]') / df['subject'].str.len()
df.head()

,subject,from,date,to,label,thread,name,date_utc,time,hour,month,num_qm,num_em,hashtag,prop_upper
0,Complete your setup with the Slack desktop app,"""Slack"" <no-reply@email.slackhq.com>",2021-01-21 11:02:05-06:00,<tanishkumar@berkeley.edu>,"Inbox,Important,Category Promotions,Unread",1.689517e+18,Slack,2021-01-21 17:02:05+00:00,11:02:05,11,1,0,0,0,0.043478
1,General Registration is now open for Winter/Sp...,Mission College <nadler@wvm.edu>,2020-12-07 13:45:42-05:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.685446e+18,Mission College,2020-12-07 18:45:42+00:00,13:45:42,13,12,0,1,0,0.078431
2,A global bank that doubles as a cybersecurity ...,"""Tricia at WayUp"" <info@bb3.wayup.com>",2020-12-08 02:19:45+00:00,"""Tanish Kumar"" <tanishkumar@berkeley.edu>","Inbox,Opened,Category Promotions",1.685475e+18,Tricia at WayUp,2020-12-08 02:19:45+00:00,02:19:45,2,12,1,1,0,0.028571
3,The next biggest thing,DailyDropout.fyi <dailydropout@substack.com>,2021-01-29 23:09:33+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.690265e+18,DailyDropout.fyi,2021-01-29 23:09:33+00:00,23:09:33,23,1,0,0,0,0.045455
4,Read free for 30 days,Scribd <hello@hello.scribd.com>,2021-02-18 08:51:55+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.692022e+18,Scribd,2021-02-18 08:51:55+00:00,08:51:55,8,2,0,0,0,0.047619


In [4]:
# The number of digits used in the whole string
df['num_digit'] = df['subject'].str.count(r'[0-9]')
df.loc[df['num_digit'] > 0].head()

,subject,from,date,to,label,thread,name,date_utc,time,hour,month,num_qm,num_em,hashtag,prop_upper,num_digit
4,Read free for 30 days,Scribd <hello@hello.scribd.com>,2021-02-18 08:51:55+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.692022e+18,Scribd,2021-02-18 08:51:55+00:00,08:51:55,8,2,0,0,0,0.047619,2
7,$32K Scholarship Giveaway Expires TONIGHT!,Nick Freud <nfreud@campusreel.org>,2021-01-31 23:09:39+00:00,Tanish Kumar <tanishkumar@berkeley.edu>,"Inbox,Opened,Category Promotions",1.690446e+18,Nick Freud,2021-01-31 23:09:39+00:00,23:09:39,23,1,0,1,0,0.261905,2
9,Drop #5: TuMeke - Computer Vision for Safer Fa...,DailyDropout.fyi <dailydropout@substack.com>,2020-10-21 16:01:17+00:00,tanishkumar@berkeley.edu,"Inbox,Important,Opened,Category Promotions",1.681178e+18,DailyDropout.fyi,2020-10-21 16:01:17+00:00,16:01:17,16,10,0,0,0,0.132075,1
10,Happy 2021!! Reward yourself with helpful star...,Harness <hello@joinharness.com>,2021-01-05 19:06:20+00:00,tanishkumar@berkeley.edu,"Inbox,Opened,Category Promotions",1.688075e+18,Harness,2021-01-05 19:06:20+00:00,19:06:20,19,1,0,3,0,0.027778,4
12,Jobs are not Compromised- NEW OPENINGS 9th May,Latest Jobs In 2020 <newsletter@latestjobsingo...,2020-05-10 23:11:11+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.666347e+18,Latest Jobs In 2020,2020-05-10 23:11:11+00:00,23:11:11,23,5,0,0,0,0.304348,1


In [5]:
# The number of fully capitalized words in the whole string
df['num_full_upper'] = df['subject'].str.count(r'\b[A-Z][A-Z]+\b')
df.loc[df['num_full_upper'] > 0].head()

,subject,from,date,to,label,thread,name,date_utc,time,hour,month,num_qm,num_em,hashtag,prop_upper,num_digit,num_full_upper
7,$32K Scholarship Giveaway Expires TONIGHT!,Nick Freud <nfreud@campusreel.org>,2021-01-31 23:09:39+00:00,Tanish Kumar <tanishkumar@berkeley.edu>,"Inbox,Opened,Category Promotions",1.690446e+18,Nick Freud,2021-01-31 23:09:39+00:00,23:09:39,23,1,0,1,0,0.261905,2,1
12,Jobs are not Compromised- NEW OPENINGS 9th May,Latest Jobs In 2020 <newsletter@latestjobsingo...,2020-05-10 23:11:11+00:00,tanishkumar@berkeley.edu,"Inbox,Category Promotions,Unread",1.666347e+18,Latest Jobs In 2020,2020-05-10 23:11:11+00:00,23:11:11,23,5,0,0,0,0.304348,1,2
13,CS 61A Exam Proctoring Link & PDF,CS 61A Exam Platform <cs61a@berkeley.edu>,2020-12-11 19:24:56+00:00,tanishkumar@berkeley.edu,"Inbox,Important,Category Promotions,Unread",1.685811e+18,CS 61A Exam Platform,2020-12-11 19:24:56+00:00,19:24:56,19,12,0,0,0,0.272727,2,2
15,"Tanish, Reminder, $32K scholarship expires TON...","""Nick Freud"" <nfreud@campusreel.org>",2020-09-30 19:31:21+00:00,"""Tanish Kumar"" <tanishkumar@berkeley.edu>","Inbox,Category Promotions,Unread",1.679288e+18,Nick Freud,2020-09-30 19:31:21+00:00,19:31:21,19,9,0,0,0,0.200000,2,1
18,Ritvik Pandey mentioned you in Calculate S&P 5...,"""Notion"" <notify@mail.notion.so>",2020-08-12 08:12:02+00:00,tanishkumar@berkeley.edu,"Inbox,Opened,Category Promotions",1.674806e+18,Notion,2020-08-12 08:12:02+00:00,08:12:02,8,8,0,0,0,0.078788,5,3


In [6]:
#The number of words with 3 and above repeated letters in the whole string
df['num_repeated'] = df['subject'].str.count(r'[A-Za-z]([A-Za-z])\1{2,}')
df.loc[df['num_repeated'] > 0].head()

,subject,from,date,to,label,thread,name,date_utc,time,hour,month,num_qm,num_em,hashtag,prop_upper,num_digit,num_full_upper,num_repeated
205,We're baaaack!,"""WayUp"" <info@bb3.wayup.com>",2020-12-04 01:23:42+00:00,"""Tanish Kumar"" <tanishkumar@berkeley.edu>","Inbox,Category Promotions,Unread",1.685109e+18,WayUp,2020-12-04 01:23:42+00:00,01:23:42,1,12,0,1,0,0.071429,0,0,1
507,Happy Friyayyyyyyyy!! Shopping will make you f...,"""Tophatter"" <noreply@tophatter.com>",2021-02-12 14:51:14+00:00,NOORGILL08@GMAIL.COM,"Inbox,Category Promotions",1.691501e+18,Tophatter,2021-02-12 14:51:14+00:00,14:51:14,14,2,0,2,0,0.049180,0,0,1
1410,Pssst - Public Goods still on your mind?,"""Influenster"" <news@i.influenster.com>",2020-12-14 20:47:43+00:00,noorgill08@gmail.com,"Inbox,Category Promotions",1.686088e+18,Influenster,2020-12-14 20:47:43+00:00,20:47:43,20,12,1,0,0,0.075000,0,0,1
1450,omg yessss!,ShoeDazzle <stylists@email.shoedazzle.com>,2020-12-19 14:58:59-05:00,noorgill08@gmail.com,"Inbox,Category Promotions",1.686538e+18,ShoeDazzle,2020-12-19 19:58:59+00:00,14:58:59,14,12,0,1,0,0.000000,0,0,1
1627,We're obsesssssed with these! All boots = 2 fo...,ShoeDazzle <stylists@email.shoedazzle.com>,2020-12-07 11:06:49-05:00,noorgill08@gmail.com,"Archived,Opened,Category Promotions",1.685436e+18,ShoeDazzle,2020-12-07 16:06:49+00:00,11:06:49,11,12,0,1,0,0.037037,5,0,1
